In [17]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats


def log_target(x):
    return np.log(0.5 * stats.norm.pdf(x, loc=4., scale=1.0) + 0.5 * stats.norm.pdf(x, loc=-4., scale=1.0)
                  )

def q_hat_random_walk_mh_proposal(x):
    return x + 3.0 * np.random.normal(loc=0., scale=1.0)

def log_q(xp, x):
    return stats.norm.logpdf(xp, loc=x, scale=3.0)

def default_monte_carlo_estimator(h, x0, q_hat, log_q, log_target, n_chain):
    s = 0.
    x = x0
    for i in range(n_chain):
        xp = q_hat(x)
        accept = np.exp(log_target(xp) + log_q(xp, x) - log_target(x) - log_q(x, xp))
        if np.random.uniform() < accept:
            x = xp
        s += h(x)
    return s/(n_chain + 1.)

def h(x):
    if x >= 3.0:
        return 1.0
    else:
        return 0.0

def maximal_coupling(p_hat, q_hat, log_p, log_q, eta=1.0):
    log_eta = np.log(eta)
    X = p_hat()
    W = - np.random.exponential()
    if W <= np.minimum(log_eta, log_q(X)-log_p(X)):
        return X, X, True
    else:
        while True:
            Ystar = q_hat()
            Wstar = -np.random.exponential()
            if Wstar > log_eta + log_p(Ystar) - log_q(Ystar):
                break
        return X, Ystar, False


def unbiased_monte_carlo_estimator(h, x0, y0, q_hat, log_q, log_target, k, m, eta=1.0, lag=1, max_iter=1e6):
    s = 0.
    mcmc = 0.
    bias_correction = 0.
    x = x0
    y = y0
    def v(t):
        return np.floor((t-k)/lag) - np.ceil(np.maximum(lag, t-m)/lag) + 1
    for time in range(lag):
        xp = q_hat(x)
        accept = np.exp(log_target(xp) + log_q(xp, x) - log_target(x) - log_q(x, xp))
        if np.random.uniform() < accept:
            x = xp
    is_coupled = False
    for time in range(lag+1, min(max_iter, m+1)):
        x_prop, y_prop, _ = maximal_coupling(
            p_hat=lambda: q_hat(x),
            q_hat=lambda: q_hat(y),
            log_p=lambda z: log_q(x, z),
            log_q=lambda z: log_q(y, z)
        )
        W = -np.random.exponential()
        accept_X = W <= log_target(x_prop) + log_q(x_prop, x) - log_target(x) - log_q(x, x_prop)
        accept_Y = W <= log_target(y_prop) + log_q(y_prop, y) - log_target(y) - log_q(y, y_prop)
        if accept_X:
            x = x_prop
        if accept_Y:
            y = y_prop
        is_coupled = (accept_X & accept_Y & _) | is_coupled
        print(is_coupled)
        print(x)
        print(y)
        mcmc += h(x)
        bias_correction += v(time) * (h(x)-h(y))
    s = (mcmc + bias_correction)/(m-k+1)
    return s, is_coupled


In [11]:
default_monte_carlo_estimator(h, 0.0, q_hat_random_walk_mh_proposal, log_q, log_target, 100000)

0.42248577514224855

In [24]:
unbiased_monte_carlo_estimator(h, np.random.uniform(), np.random.uniform(), q_hat_random_walk_mh_proposal, log_q, log_target, 1, 10)

False
-3.984150269521229
2.2016574134621365
False
-2.9195138460590866
2.2016574134621365
True
-1.8416997831197903
-1.8416997831197903
True
-1.8416997831197903
-1.8416997831197903
True
-2.012894572278365
-2.012894572278365
True
-2.470024708583208
-2.470024708583208
True
-4.447244743709434
-4.447244743709434
True
-5.6998639413553605
-5.6998639413553605
True
-6.4781038509546915
-6.4781038509546915


(0.0, True)